In [1]:
# PYTHON IMPORTS
import os
import copy
from tqdm.notebook import trange, tqdm

# IMAGE IMPORTS 
from PIL import Image
import cv2

# DATA IMPORTS 
import random
import h5py
import netCDF4 as nc
import numpy as np

# PLOTTING
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# NEURAL NETWORK
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from torchvision.transforms import ToPILImage, GaussianBlur
from torchvision.transforms import Compose, RandomCrop, ToTensor, Normalize
import torch.optim.lr_scheduler as lr_scheduler
import torchvision.models as models

# MY OWN CLASSES
from TileLocator import *

In [4]:
input_folder = "data/TileLocator/in"
val_folder = "data/TileLocator/in"
target_folder = "data/TileLocator/out"
batch_size = 2

transform = transforms.Compose([
    transforms.RandomRotation(degrees=180),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

tensor = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = SegmentationDataset(input_folder, target_folder, transform=transform, crop=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

val_dataset = SegmentationDataset(val_folder, target_folder, transform=transform, crop=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=0)

loaders = {'train' : train_loader, 'val' : val_loader}

C:\Users\franc\anaconda3\envs\deeplearning\lib\site-packages\PIL\Image.py:2921: DecompressionBombWarning: Image size (151368000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [5]:
result = train_dataset[0]

In [6]:
result[1].shape

torch.Size([1024, 1024])

In [7]:
def train(model, dataloaders, num_epochs=50, output_dir='data/TileLocator/intermediate_outputs', learning_rate=5e-4):
    device = torch.device("cuda:0")
    weights = torch.tensor([1, 120]).float().to(device)
    criterion = nn.CrossEntropyLoss(weight=weights, 
        reduction="mean")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    learning_rate_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.97)
    
    
    
    model = model.to(device)
    # since = time.time()
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        for phase in ['train', 'val']: 
            if phase == 'train':
                model.train()
                repeats = range(32)
            else:
                model.eval()
                repeats = range(2)

            running_loss = 0.0
            
            outputs_folder = os.path.join(output_dir, phase)
            if not os.path.exists(outputs_folder):
                os.makedirs(outputs_folder)
                
            curr_loss = 0
            
            # Iterate over data.
            for rep_id in tqdm(repeats):
                if rep_id % 8 == 0 and rep_id != 0:
                    print(f"{curr_loss:.4e} {curr_loss/8:.4e}")
                    curr_loss = 0
                for inputs, labels, filenames in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    
                    loss = criterion(outputs, labels) 
                    curr_loss += loss
                    if rep_id % 8 == 0 and phase is 'train' and rep_id != 0:
                        prob_img_or = outputs.detach().cpu()
                        minputs = inputs.detach().cpu().numpy()
                        
                        if prob_img_or.ndim == 3:
                            prob_img_or = prob_img_or.unsqueeze(0)
                            
                        prob_img_or = prob_img_or.numpy()
                        
                        for i in range(len(outputs)):
                            filename = filenames[i]
                            prob_img = prob_img_or[i, 1, :, :]
                            prob_img = (prob_img * 255).astype(np.uint8)
                            prob_img = Image.fromarray(np.squeeze(prob_img))
                            prob_img.save(os.path.join(outputs_folder, f"{rep_id}_{filename}"))
                            myinp = Image.fromarray(np.uint8(minputs[i, 0, :, :] * 255))
                            myinp.save(os.path.join(outputs_folder, f"{rep_id}_{filename[:-3]}_inp.png"))
                    if phase is 'train':
                        loss.backward()
                        optimizer.step()
                    else:
                        prob_img_or = outputs.detach().cpu() 
                        minputs = inputs.detach().cpu().numpy()
                        if prob_img_or.ndim == 3:
                            prob_img_or = prob_img_or.unsqueeze(0)
                        prob_img_or = prob_img_or.numpy()    
                        
                        for i in range(len(outputs)):
                            filename = filenames[i]
                            prob_img = prob_img_or[i, 1, :, :]
                            prob_img = (prob_img * 255).astype(np.uint8)
                            prob_img = Image.fromarray(np.squeeze(prob_img))
                            prob_img.save(os.path.join(outputs_folder, f"{rep_id}_{filename}"))
                            myinp = Image.fromarray(np.uint8(minputs[i, 0, :, :] * 255))
                            myinp.save(os.path.join(outputs_folder, f"{rep_id}_{filename[:-3]}_inp.png"))
                    running_loss += loss.item()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            if phase == 'val' and epoch_loss < best_acc:
                best_model_wts = copy.deepcopy(model.state_dict())
        # Update the learning rate scheduler after each epoch
        learning_rate_scheduler.step()
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']}")
        # Save the model and optimizer states
        if epoch % 10 == 0:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
            }, 'data/TileLocator/checkpoint_071323.pth')
            
            torch.save(model, r"data/TileLocator/071323.pth")
        
    return model

In [ ]:
model = RectangleClass()
model = train(model, loaders, num_epochs=500, learning_rate=1e-3)

Epoch 1/500
----------


  0%|          | 0/32 [00:00<?, ?it/s]

C:\Users\franc\anaconda3\envs\deeplearning\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
C:\Users\franc\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:65: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


5.3897e+00 6.7372e-01
4.6355e+00 5.7944e-01
3.9255e+00 4.9069e-01
train Loss: 8.7299


  0%|          | 0/2 [00:01<?, ?it/s]

...

2.6968e+00 3.3710e-01
2.6369e+00 3.2961e-01
2.6764e+00 3.3455e-01
train Loss: 5.3616


  0%|          | 0/2 [00:00<?, ?it/s]

val Loss: 0.6660
Learning Rate: 0.0006938423609954377
Epoch 122/500
----------


  0%|          | 0/32 [00:00<?, ?it/s]

2.6469e+00 3.3086e-01
